# Complete Workflow for Project

## 1. User and Content Creator Registration

### Frontend (Chrome Extension)


#### Sign-Up Interface
- Users and content creators register using the Chrome extension.
- The registration form includes basic fields (name, email, password) and role selection (either **User** or **Content Creator**).
- Registration data is sent to the backend.
    

### Backend (FastAPI)

In [ ]:

# Implementation of the user and content creator registration using FastAPI
from fastapi import FastAPI, Depends
from pydantic import BaseModel

app = FastAPI()

# A Pydantic model for user data
class User(BaseModel):
    username: str
    email: str
    password: str
    role: str  # Either 'user' or 'content_creator'

# Mock database
db = {"users": []}

# Register endpoint
@app.post("/register")
async def register(user_data: User):
    db['users'].append(user_data.dict())  # Add the user to the mock database
    return {"message": "Registration successful", "user": user_data.dict()}

# Dependency to simulate getting the current user
def get_current_user():
    return db["users"][-1]  # For simplicity, return the last registered user


## 2. Content Creator Workflow

### Frontend (Chrome Extension)


#### Intercepting Post Attempts
- The Chrome extension monitors **post attempts** on supported platforms (e.g., WordPress, Medium).
- If a content creator tries to post, the extension intercepts the request and checks if the user is logged in.
- If not logged in, the extension **prompts for login** or **blocks the post attempt**.

Here's the JavaScript logic for Chrome extension:
```javascript
chrome.webRequest.onBeforeRequest.addListener(
  function(details) {
    if (!localStorage.getItem("authToken")) {
      chrome.runtime.sendMessage({ action: "promptLogin" });
      return { cancel: true };  // Block post attempt
    }
  },
  { urls: ["<all_urls>"] },
  ["blocking"]
);
```
    

### Backend (FastAPI)

In [ ]:

# Endpoint to track content creation (post submission)
class PostDetails(BaseModel):
    platform: str
    content: str

@app.post("/track_post")
async def track_post(post_details: PostDetails, current_user: User = Depends(get_current_user)):
    # Simulating saving the post and tracking it for verification
    post_info = {
        "creator": current_user.username,
        "platform": post_details.platform,
        "content": post_details.content,
        "status": "under_review"
    }
    return {"message": "Post tracked and under review", "post_info": post_info}


## 3. User Workflow

### Frontend (Chrome Extension)


#### Interactive Pop-Up Interface
- Users can select content to verify (image, video, or text) via the pop-up interface.

JavaScript logic for content verification:
```javascript
document.getElementById('verifyButton').addEventListener('click', () => {
    const selectedContent = getSelectedContent();
    
    fetch('http://127.0.0.1:8000/verify', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ content: selectedContent }),
    })
    .then(response => response.json())
    .then(data => {
        document.getElementById('result').innerText = `Result: ${data.result}`;
    });
});
```
    

### Backend (FastAPI)

In [ ]:

# Endpoint for content verification
class Content(BaseModel):
    content: str

@app.post("/verify")
async def verify_content(content: Content, current_user: User = Depends(get_current_user)):
    # Mock verification logic
    if len(content.content) < 10:
        return {"result": "Content too short, possibly illegitimate"}
    
    return {"result": "Content is legitimate"}


## 4. Detecting Malicious Intent from Users

### Frontend (Chrome Extension)


#### Detecting Malicious Content
- The extension checks for malicious content before submitting it for verification.
- If detected, the user gets a warning.

```javascript
document.getElementById('verifyButton').addEventListener('click', () => {
    const selectedContent = getSelectedContent();
    
    if (isMaliciousContent(selectedContent)) {
        alert("Malicious content detected. Aborting verification.");
        return;
    }

    fetch('http://127.0.0.1:8000/verify', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ content: selectedContent }),
    })
    .then(response => response.json())
    .then(data => {
        document.getElementById('result').innerText = `Result: ${data.result}`;
    });
});
```
    

### Backend (FastAPI)

In [ ]:

# Mock malicious content detection logic
def detect_malicious_content(content):
    malicious_keywords = ["spam", "malicious", "offensive"]
    return any(keyword in content for keyword in malicious_keywords)

@app.post("/verify")
async def verify_content(content: Content, current_user: User = Depends(get_current_user)):
    if detect_malicious_content(content.content):
        log_warning(current_user.username)
        return {"result": "Malicious content detected"}

    return {"result": "Content is legitimate"}


## 5. Malicious Content Reporting and Verification

### Frontend (Chrome Extension)


#### User Reporting of Malicious Content
Users can report inappropriate content, which is then verified.

JavaScript logic for reporting malicious content:
```javascript
document.getElementById('reportButton').addEventListener('click', () => {
    const reportedContent = getReportedContent();
    
    fetch('http://127.0.0.1:8000/report', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ content: reportedContent }),
    })
    .then(response => response.json())
    .then(data => {
        alert('Report submitted successfully.');
    });
});
```
    

### Backend (FastAPI)

In [ ]:

# Mock malicious content reporting and verification
@app.post("/report")
async def report_content(content: Content, current_user: User = Depends(get_current_user)):
    if detect_malicious_content(content.content):
        handle_malicious_content(content.content)
        return {"message": "Content verified as malicious. Action taken."}
    
    log_report(current_user.username, content.content)
    return {"message": "Content reported. Under review."}

# Placeholder function for handling malicious content
def handle_malicious_content(content):
    print(f"Action taken against content: {content}")


## 6. Content Creator Penalties

### Backend (FastAPI)

In [ ]:

# Implementing penalties for content creators
MALICIOUS_THRESHOLD = 3  # Threshold for penalizing content creators

def calculate_malicious_score(creator_id):
    # Mock function that calculates a malicious score for a content creator
    return 4  # Returning a score greater than threshold

def restrict_content_creator(creator_id):
    print(f"Content creator {creator_id} restricted from posting for 30 days")

@app.post("/handle_penalty")
async def handle_penalty(creator_id: str):
    malicious_score = calculate_malicious_score(creator_id)
    
    if malicious_score > MALICIOUS_THRESHOLD:
        restrict_content_creator(creator_id)
        return {"message": "Content creator restricted for malicious content."}
    
    return {"message": "Content creator warned"}
